#Google Colaboratory RDP Server

#Method 1

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@title **RDP**

! wget https://raw.githubusercontent.com/VivekDomadia/Google-Colaboratory-RDP-Server/master/RDP.sh &> /dev/null
! chmod +x RDP.sh
! ./RDP.sh

#Method 2

In [ ]:
#@title **Create User**
#@markdown Enter Username and Password

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print("User Created and Configured")

In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

#@markdown  Visit <a href="http://remotedesktop.google.com/headless" target='_blank'>http://remotedesktop.google.com/headless</a> and Copy the command after authentication
CRP = "" #@param {type:"string"}

#@markdown Enter a pin more or equal to 6 digits
Pin = 123456 #@param {type: "integer"}

import os
import subprocess

class CRD:
  def __init__(self):
    os.system("apt update")
    self.installCRD()
    self.installDesktopEnvironment()
    self.installGoogleChorme()
    self.finish()
    os.system("apt upgrade")

  @staticmethod
  def installCRD():
    print("Installing Chrome Remote Desktop")
    subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
    subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
    subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

  @staticmethod
  def installDesktopEnvironment():
    print("Installing Desktop Environment")
    os.system("export DEBIAN_FRONTEND=noninteractive")
    os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
    os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
    os.system("apt remove --assume-yes gnome-terminal")
    os.system("apt install --assume-yes xscreensaver")
    os.system("systemctl disable lightdm.service")

  @staticmethod
  def installGoogleChorme():
    print("Installing Google Chrome")
    subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
    subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
    subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

  @staticmethod
  def finish():
    print("Finalizing")
    os.system(f"adduser {username} chrome-remote-desktop")
    command = f"{CRP} --pin={Pin}"
    os.system(f"su - {username} -c '{command}'")
    os.system("service chrome-remote-desktop start")
    print("Finished Succesfully")
    print("Please check https://remotedesktop.google.com/access/")

try:
  if username:
    if CRP == "":
      print("Please enter authcode from the given link")
    elif len(str(Pin)) < 6:
      print("Enter a pin more or equal to 6 digits")
    else:
      CRD()
except NameError as e:
  print("username variable not found")
  print("Create a User First")

#Mount Drive

In [ ]:
#@title **Google Drive Mount**
#@markdown Mounted at `/home/USERNAME/drive` folder
#@markdown (If `USERNAME` variable not defined then use `/content/drive` as default).


try:
  if username:
    mountpoint = "/home/"+username+"/drive"
  if username == 'root':
    mountpoint = '/content/drive'  
except NameError:
  mountpoint = '/content/drive'

from google.colab import drive
drive.mount(mountpoint)

# SSH Method

In [ ]:
#@title **SSH**

#@markdown Copy authtoken from <a href='https://dashboard.ngrok.com/auth' target='_blank'>https://dashboard.ngrok.com/auth</a>
NGROK_TOKEN = "" #@param {type:'string'}

!pip install colab_ssh &> /dev/null
from colab_ssh import launch_ssh
from IPython.display import clear_output

def runNGROK():
  launch_ssh(NGROK_TOKEN, password)
  clear_output()

  print("ssh", username, end='@')
  ! curl -s http://localhost:4040/api/tunnels | python3 -c \
          "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"
  print('Username:',username)
  print('Password:', password)

try:
  if username:
    pass
  elif password:
    pass
except NameError:
  username='root'
  password='root'

if NGROK_TOKEN:
  try:
    runNGROK()
  except Exception as e:
    print(e)
    print('Your Token',NGROK_TOKEN)
else:
  print("No NGROK_TOKEN Found, Please enter it")

#Colab Shutdown

In [ ]:
#@title **Colab Shutdown**

#@markdown To Kill NGROK Tunnel
KILL_NGROK = False #@param {type:'boolean'}

#@markdown To Unmount GDrive
UNMOUNT_GDRIVE = False #@param {type:'boolean'}

#@markdown To Sleep Colab
Sleep = 0 #@param {type:"integer"}

if KILL_NGROK:
  !killall ngrok

if UNMOUNT_GDRIVE:
  from google.colab import drive
  drive.flush_and_unmount()

if Sleep:
  from time import sleep
  sleep(Sleep)